In [1]:
!pip install pytorch-tabnet
!pip install wandb

In [2]:
import argparse
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder

import torch
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.metrics import Metric

import wandb
wandb.init(project="DACON_235877")

parser = argparse.ArgumentParser(description='TabNet')
parser.add_argument('--validation_split', default=0.1, type=float)
parser.add_argument('--n_d', default=8, type=int) # 8~64
parser.add_argument('--n_a', default=8, type=int) # n_d=n_a
parser.add_argument('--n_steps', default=3, type=int) # 3~10
parser.add_argument('--gamma', default=1.3, type=float) # 1.0~2.0
parser.add_argument('--batch_size', default=64, type=int) # 1024
parser.add_argument('--virtual_batch_size', default=8, type=int) # 128
parser.add_argument('--mask_type', default="sparsemax", type=str) # sparsemax or entmax
parser.add_argument('--pretraining_ratio', default=0.8, type=float) # 0~1
parser.add_argument('--seed', default=1011, type=int)
args = parser.parse_args('')

wandb.config.update(args)

random_seed=args.seed

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

train = pd.read_csv("/content/drive/MyDrive/DACON-Basic/235877_전복 나이 예측 경진대회/data/train.csv")
test = pd.read_csv("/content/drive/MyDrive/DACON-Basic/235877_전복 나이 예측 경진대회/data/test.csv")

train.head()

wandb: Currently logged in as: gnoeyheat (use `wandb login --relogin` to force relogin)


,id,Gender,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Target
0,1,M,0.605,0.470,0.115,1.1140,0.3925,0.2910,0.3100,15
1,2,I,0.430,0.315,0.095,0.3780,0.1750,0.0800,0.1045,8
2,3,I,0.580,0.490,0.195,1.3165,0.5305,0.2540,0.4100,18
3,4,M,0.535,0.405,0.175,1.2705,0.5480,0.3265,0.3370,13
4,5,I,0.310,0.235,0.090,0.1270,0.0480,0.0310,0.0400,6


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              1253 non-null   int64  
 1   Gender          1253 non-null   object 
 2   Lenght          1253 non-null   float64
 3   Diameter        1253 non-null   float64
 4   Height          1253 non-null   float64
 5   Whole Weight    1253 non-null   float64
 6   Shucked Weight  1253 non-null   float64
 7   Viscra Weight   1253 non-null   float64
 8   Shell Weight    1253 non-null   float64
 9   Target          1253 non-null   int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 98.0+ KB


In [4]:
categorical_columns = []
categorical_dims =  {}
for col in train.columns[train.dtypes == object]:
    print(col, train[col].nunique())
    l_enc = LabelEncoder()
    train[col] = l_enc.fit_transform(train[col].values)
    test[col] = l_enc.fit_transform(test[col].values)
    categorical_columns.append(col)
    categorical_dims[col] = len(l_enc.classes_)

target = 'Target'
unused_feat = ['id']
features = [col for col in train.columns if col not in unused_feat+[target]] 

cat_idxs = [i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

train.info()

Gender 3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              1253 non-null   int64  
 1   Gender          1253 non-null   int64  
 2   Lenght          1253 non-null   float64
 3   Diameter        1253 non-null   float64
 4   Height          1253 non-null   float64
 5   Whole Weight    1253 non-null   float64
 6   Shucked Weight  1253 non-null   float64
 7   Viscra Weight   1253 non-null   float64
 8   Shell Weight    1253 non-null   float64
 9   Target          1253 non-null   int64  
dtypes: float64(7), int64(3)
memory usage: 98.0 KB


In [5]:
## Data Preprocessing

X = train.drop(["id", "Target"], axis=1)
y = train.Target

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=args.validation_split, random_state=random_seed)
X_test = test.drop(["id"], axis=1)

X_train = X_train.values
X_valid = X_valid.values
X_test = X_test.values

y_train = y_train.values.reshape(-1, 1)
y_valid = y_valid.values.reshape(-1, 1)

X_train.shape, X_valid.shape, X_test.shape, y_train.shape, y_valid.shape

((1127, 8), (126, 8), (2924, 8), (1127, 1), (126, 1))

In [6]:
class NMAE(Metric):
    def __init__(self):
        self._name = "nmae"
        self._maximize = False

    def __call__(self, y_true, y_score):
        target_idx = np.where(y_true!=0)
        true = y_true[target_idx]
        pred = y_score[target_idx]
        score = np.mean(np.abs(true-pred)/true)
        return score

* Supervised learning

In [7]:
clf = TabNetRegressor(
    n_d=args.n_d, n_a=args.n_a, n_steps=args.n_steps, gamma=args.gamma,
    cat_idxs = cat_idxs,
    cat_dims = cat_dims,
    seed=random_seed,
    optimizer_fn=torch.optim.Adam,
    mask_type=args.mask_type
    )

clf.fit(
  X_train, y_train, 
  eval_set=[(X_valid, y_valid)],
  eval_metric=[NMAE],
  max_epochs=200,
  patience=15,
  loss_fn=torch.nn.L1Loss(),
  batch_size=args.batch_size,
  virtual_batch_size=args.virtual_batch_size
)

preds = clf.predict(X_test)

Device used : cpu
epoch 0  | loss: 5.04491 | val_0_nmae: 0.40994 |  0:00:01s
epoch 1  | loss: 2.66037 | val_0_nmae: 0.29465 |  0:00:01s
epoch 2  | loss: 2.1387  | val_0_nmae: 0.23129 |  0:00:02s
epoch 3  | loss: 2.01033 | val_0_nmae: 0.25492 |  0:00:02s
epoch 4  | loss: 2.00552 | val_0_nmae: 0.24977 |  0:00:03s
epoch 5  | loss: 1.95909 | val_0_nmae: 0.30572 |  0:00:04s
epoch 6  | loss: 1.89941 | val_0_nmae: 0.2463  |  0:00:05s
epoch 7  | loss: 1.92558 | val_0_nmae: 0.27138 |  0:00:06s
epoch 8  | loss: 1.85371 | val_0_nmae: 0.25617 |  0:00:06s
epoch 9  | loss: 1.88599 | val_0_nmae: 0.22323 |  0:00:07s
epoch 10 | loss: 1.87407 | val_0_nmae: 0.23189 |  0:00:08s
epoch 11 | loss: 1.9257  | val_0_nmae: 0.21743 |  0:00:10s
epoch 12 | loss: 1.8799  | val_0_nmae: 0.24066 |  0:00:11s
epoch 13 | loss: 1.82271 | val_0_nmae: 0.23632 |  0:00:12s
epoch 14 | loss: 1.79053 | val_0_nmae: 0.25005 |  0:00:12s
epoch 15 | loss: 1.85365 | val_0_nmae: 0.23461 |  0:00:14s
epoch 16 | loss: 1.81242 | val_0_nmae:

* Semi-supervised pre-training

In [8]:
unsupervised_model = TabNetPretrainer(
    n_d=args.n_d, n_a=args.n_a, n_steps=args.n_steps, gamma=args.gamma,
    cat_idxs = cat_idxs,
    cat_dims = cat_dims,
    seed=random_seed,
    optimizer_fn=torch.optim.Adam,
    mask_type=args.mask_type
)

unsupervised_model.fit(
    X_train=X_train,
    eval_set=[X_valid],
    pretraining_ratio=args.pretraining_ratio,
)

clf = TabNetRegressor(
    n_d=args.n_d, n_a=args.n_a, n_steps=args.n_steps, gamma=args.gamma,
    cat_idxs = cat_idxs,
    cat_dims = cat_dims,
    seed=random_seed,
    optimizer_fn=torch.optim.Adam,
    scheduler_params={"step_size":10,
                      "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type=args.mask_type
)

clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=[NMAE],
    from_unsupervised=unsupervised_model
)

Device used : cpu
epoch 0  | loss: 146.40552| val_0_unsup_loss: 60.45023|  0:00:00s
epoch 1  | loss: 39.68282| val_0_unsup_loss: 11.18059|  0:00:00s
epoch 2  | loss: 24.93044| val_0_unsup_loss: 8.83402 |  0:00:00s
epoch 3  | loss: 11.98144| val_0_unsup_loss: 7.25275 |  0:00:00s
epoch 4  | loss: 9.08108 | val_0_unsup_loss: 8.72295 |  0:00:00s
epoch 5  | loss: 7.04899 | val_0_unsup_loss: 4.93969 |  0:00:00s
epoch 6  | loss: 5.37571 | val_0_unsup_loss: 4.57178 |  0:00:00s
epoch 7  | loss: 3.93873 | val_0_unsup_loss: 5.09128 |  0:00:00s
epoch 8  | loss: 4.26838 | val_0_unsup_loss: 2.79292 |  0:00:00s
epoch 9  | loss: 3.14727 | val_0_unsup_loss: 2.09433 |  0:00:01s
epoch 10 | loss: 2.43558 | val_0_unsup_loss: 1.88939 |  0:00:01s
epoch 11 | loss: 1.94448 | val_0_unsup_loss: 1.64595 |  0:00:01s
epoch 12 | loss: 2.00126 | val_0_unsup_loss: 1.75157 |  0:00:01s
epoch 13 | loss: 1.6772  | val_0_unsup_loss: 2.43615 |  0:00:01s
epoch 14 | loss: 1.69129 | val_0_unsup_loss: 2.62923 |  0:00:01s
epoch 

In [9]:
# saving_path_name = "./tabnet_model_test_1"
# saved_filepath = clf.save_model(saving_path_name)

# loaded_clf = TabNetRegressor()
# loaded_clf.load_model(saved_filepath)

In [10]:
submission=clf.predict(X_test).round()

sample_submission = pd.read_csv("/content/drive/MyDrive/DACON-Basic/235877_전복 나이 예측 경진대회/data/sample_submission.csv")
sample_submission.Target = submission
sample_submission.to_csv("/content/drive/MyDrive/DACON-Basic/235877_전복 나이 예측 경진대회/submission.csv",index=False)